# Machine Learning
### Monte Carlo and expectation
The problem is to estimate $E[g(X)]=\int g(x) p(x) dx$, where $X \sim p(x)$. Depending on whether we can sample from **p(x)** or not, we get two versions of Monte Carlo for expectation:
<br>a. **Direct Sampling** (if you can sample from $p(x)$):
1. Generate $n$ samples from $p(x)$: $x_i\sim p(x)$
2. Compute $g(x_i)$ for each sample.
3. Average:
$E[g(X)]\approx\frac{1}{n}\sum_{i=1}^n g(x_i)$

**Reminder:** Standard Error is:
- $SE=\frac{\sigma}{\sqrt{n}}$, where $\sigma^2=Var(g(X))$

b.1. **Importance Sampling** (if you can't sample from $p(x)$ directly):
1. Choose a proposal distribution $q(x)$ that is easy to sample from
2. Generate $n$ samples from $q(x)$: $x_i\sim q(x)$
3. Weight each sample by $w_i=p(x_i)/q(x_i)$
4. Compute weighted average: $E[g(X)]≈\frac{1}{n}\sum_{i=1}^n g(x_i)⋅\frac{p(x_i)}{q(x_i)}$

b.2. **Self-Normalized Importance Sampling** (when p is unnormalized): Assumption is that we only know $\tilde{p}(x)$ where $p(x)=\frac{\tilde{p}(x)}{Z}$, and $Z=\int \tilde{p}(x) dx$ is unknown.
<br>$E_p[g(X)]≈\frac{\sum_{i=1}^n w_i\cdot g(x_i)}{\sum_{i=1}^n w_i}$, $w_i=\frac{\tilde{p}(x_i)}{q(x_i)}$, $x_i \sim q$


<hr>

In the following:
- We first give the Python code for **direct sampling** from scratch. Then, we use it for three exampples.
- Finally, we give the code for **Importance sampling**, which supports both nromalzied and unnormlaized **p(x)**. After that, we test the code for computing an expectation with a few proposal **q(x)**.

<hr>

https://github.com/ostad-ai/Machine-Learning
<br> Explanation: https://www.pinterest.com/HamedShahHosseini/Machine-Learning/

In [1]:
import numpy as np
import matplotlib.pyplot as plt

<hr style="height:3px;background:lightblue">

# Direct Sampling

In [2]:
def direct_monte_carlo(g, sample_generator, n=10000, return_samples=False):
    """
    Method 1. Direct Monte Carlo: E[g(X)] ≈ (1/n) Σ g(X_i), X_i ~ p(x)

    Parameters:
    -----------
    g : function
        Function g(x) whose expectation we want
    sample_generator : function
        Function that returns random samples from p(x)
    n : int
        Number of samples
    return_samples : bool
        Whether to return all samples and evaluations

    Returns:
    --------
    estimate : float
        Estimate of E[g(X)]
    std_error : float
        Standard error of estimate
    (optional) samples, evaluations : arrays
        If return_samples=True
    """
    samples = []
    evaluations = []

    for _ in range(n):
        x = sample_generator()
        y = g(x)
        samples.append(x)
        evaluations.append(y)

    samples = np.array(samples)
    evaluations = np.array(evaluations)

    estimate = np.mean(evaluations)
    std_error = np.std(evaluations, ddof=1) / np.sqrt(n)

    if return_samples:
        return estimate, std_error, samples, evaluations
    else:
        return estimate, std_error

In [3]:
# Example: Estimate E[X] and Var[X] for X ~ Gaussian

# True values of mean and variance
true_mean,true_var=3,4

# Define the target distribution (normal distribution)
def sample_normal():
    """Sample from N(μ, σ)"""
    return np.random.normal(true_mean, np.sqrt(true_var))

# Estimate E[X]
def g_mean(x):
    return x

# Estimate E[X²] (for variance)
def g_squared(x):
    return x**2

# Run Monte Carlo
mean_estimate, mean_error = direct_monte_carlo(g_mean, sample_normal, n=10000)
squared_estimate, squared_error = direct_monte_carlo(g_squared, sample_normal, n=10000)

# Calculate variance estimate
variance_estimate = squared_estimate - mean_estimate**2
print(f"Estimated mean: {mean_estimate:.4f} ± {mean_error:.4f}")
print(f"Estimated variance: {variance_estimate:.4f}")
print(f"True mean: {true_mean}, True variance: {true_var}")

Estimated mean: 2.9973 ± 0.0199
Estimated variance: 3.9499
True mean: 3, True variance: 4


In [4]:
# Estimate P(X > 1) for X ~ N(0,1)
def sample_standard_normal():
    return np.random.normal(0, 1)

def indicator_greater_than_1(x):
    """Indicator function: 1 if x > 1, 0 otherwise"""
    return 1 if x > 1 else 0

# Probability is just an expectation of indicator function
prob_estimate, prob_error = direct_monte_carlo(
    indicator_greater_than_1, 
    sample_standard_normal, 
    n=50000
)

print(f"Estimated P(X > 1): {prob_estimate:.4f} ± {prob_error:.4f}")
print(f"True P(X > 1): {0.1587:.4f}")

Estimated P(X > 1): 0.1596 ± 0.0016
True P(X > 1): 0.1587


In [5]:
# Example: Estimate E[sin(X) * exp(-X²)] for X ~ Uniform(0, π)
def sample_uniform():
    return np.random.uniform(0, np.pi)

def complex_function(x):
    return np.sin(x) * np.exp(-x**2)

estimate, error = direct_monte_carlo(complex_function, sample_uniform, n=100000)
print(f"Estimated E[sin(X)exp(-X²)]: {estimate:.6f} ± {error:.6f}")

# Compare with numerical integration
import scipy.integrate as integrate
true_value, _ = integrate.quad(lambda x: complex_function(x) * (1/np.pi), 0, np.pi)
print(f"True value (numerical integration): {true_value:.6f}")

Estimated E[sin(X)exp(-X²)]: 0.135224 ± 0.000458
True value (numerical integration): 0.135103


<hr style="height:3px;background:lightblue">

# Importance Sampling


In [6]:
def importance_sampling(g, p, sample_q, q_pdf, n=10000, 
                      normalized_p=True, return_weights=False):
    """
    Method 2. Importance Sampling for estimating E_p[g(X)]
    
    Parameters:
    -----------
    g : function
        Function g(x) whose expectation we want
    p : function
        Target distribution PDF. If normalized_p=True, p must integrate to 1.
        If normalized_p=False, p can be unnormalized (proportional to true PDF).
    sample_q : function
        Function that returns random samples from q(x)
    q_pdf : function
        PDF of proposal distribution q(x) (normalized)
    n : int
        Number of samples
    normalized_p : bool
        - True: p is normalized PDF (∫p(x)dx = 1) → uses standard IS
        - False: p is unnormalized (p̃(x) ∝ true PDF) → uses self-normalized IS
    return_weights : bool
        Whether to return weights

    Returns:
    --------
    estimate : float
        Estimate of E[g(X)]
    effective_n : float
        Effective sample size
    (optional) weights : array
        If return_weights=True
    """
    # Generate samples
    samples = np.array([sample_q() for _ in range(n)])
    
    # Calculate weights: w(x) = p(x)/q(x)
    weights = p(samples) / q_pdf(samples)
    g_values = g(samples)
    
    if normalized_p:
        # Standard Importance Sampling
        # μ̂ = (1/n) Σ w_i * g(x_i)
        estimate = np.mean(weights * g_values)
        
        # Effective sample size for standard IS
        # ESS = n / (1 + CV^2) where CV = coefficient of variation of weights
        mean_w = np.mean(weights)
        if mean_w > 0:
            cv2 = np.var(weights) / (mean_w**2)
            eff_sample_size = n / (1 + cv2)
        else:
            eff_sample_size = 0.0
            
    else:
        # Self-Normalized Importance Sampling
        # μ̂ = Σ w_i * g(x_i) / Σ w_i
        weighted_sum = np.sum(weights * g_values)
        total_weight = np.sum(weights)
        
        if abs(total_weight) > 1e-12:
            estimate = weighted_sum / total_weight
            
            # Effective sample size for self-normalized IS
            # ESS = (Σ w_i)^2 / Σ w_i^2
            eff_sample_size = (total_weight**2) / np.sum(weights**2)
        else:
            estimate = 0.0
            eff_sample_size = 0.0
    
    if return_weights:
        return estimate, eff_sample_size, weights
    else:
        return estimate, eff_sample_size

In [7]:
# Example: E[e^X] where X ~ N(0,1) using importance sampling
print("\n" + "=" * 60)
print("EXAMPLE 2: Importance Sampling")
print("=" * 60)

# Target: E[e^X] where X ~ N(0,1)
# Theoretical: E[e^X] = exp(μ + σ²/2) = exp(0 + 1/2) = exp(0.5) ≈ 1.64872

g = lambda x: np.exp(x)

# Target distribution: N(0,1)
p = lambda x: np.exp(-0.5 * x**2) / np.sqrt(2*np.pi)

# Try different proposal distributions
proposals = [
    ("N(0,1)",  # Same as target (bad for demonstration)
     lambda: np.random.randn(),
     lambda x: np.exp(-0.5 * x**2) / np.sqrt(2*np.pi)),

    ("N(1,1)",  # Shifted Gaussian
     lambda: np.random.randn() + 1,
     lambda x: np.exp(-0.5 * (x-1)**2) / np.sqrt(2*np.pi)),

    ("N(0.5,1.5)",  # Different mean and variance
     lambda: np.random.randn() * 1.5 + 0.5,
     lambda x: np.exp(-0.5 * ((x-0.5)/1.5)**2) / (1.5 * np.sqrt(2*np.pi))),

    ("Laplace(0,1)",  # Heavy-tailed
     lambda: np.random.laplace(0, 1),
     lambda x: 0.5 * np.exp(-np.abs(x))),
]

true_value = np.exp(0.5)
n = 10000

print(f"\nEstimating E[e^X] where X ~ N(0,1):")
print(f"Theoretical value: {true_value:.6f}")
print(f"\nUsing n = {n} samples:")
print(f"{'Proposal':<15} {'Estimate':>12} {'Error':>12} {'Eff. n':>12} {'Rel. Error':>12}")
print("-" * 70)

for name, sample_q, q_pdf in proposals:
    estimate, eff_n = importance_sampling(
        g, p, sample_q, q_pdf, n=n
    )

    error = abs(estimate - true_value)
    rel_error = error / true_value

    print(f"{name:<15} {estimate:12.6f} {error:12.6f} "
          f"{eff_n:12.1f} {rel_error:12.6f}")

# Visualize importance sampling
n_samples = 1000

# Use N(0.5, 1.5) as proposal
proposal_name, sample_q, q_pdf = proposals[2]

estimate, eff_n, weights = importance_sampling(
    g, p, sample_q, q_pdf, n=n_samples, return_weights=True
)


EXAMPLE 2: Importance Sampling

Estimating E[e^X] where X ~ N(0,1):
Theoretical value: 1.648721

Using n = 10000 samples:
Proposal            Estimate        Error       Eff. n   Rel. Error
----------------------------------------------------------------------
N(0,1)              1.666733     0.018012      10000.0     0.010925
N(1,1)              1.648721     0.000000       4120.6     0.000000
N(0.5,1.5)          1.661398     0.012677       7766.2     0.007689
Laplace(0,1)        1.650747     0.002025       9054.1     0.001228
